In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from captum.attr import IntegratedGradients
from torchvision import transforms
from PIL import Image
import cv2

def visualize_attention_map(model, image_path, true_label, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()

    # Завантаження зображення
    image = Image.open(image_path).convert('RGB')

    # Трансформація
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Прогноз
    with torch.no_grad():
        output = torch.sigmoid(model(input_tensor)).item()

    predicted_class = 1 if output >= 0.5 else 0
    print(f"Predicted: {predicted_class} ({'Smoker' if predicted_class else 'Non-Smoker'}), Score: {output:.3f}")
    print(f"True label: {true_label} ({'Smoker' if true_label else 'Non-Smoker'})")

    # Integrated Gradients — без вказання target для скалярного виходу
    ig = IntegratedGradients(model)
    attribution, delta = ig.attribute(input_tensor, return_convergence_delta=True)

    # В обчисленнях може бути лише один канал → перевіримо
    attribution = attribution.squeeze(0).cpu().detach().numpy()
    if attribution.shape[0] == 3:
        grayscale_attribution = np.mean(attribution, axis=0)
    else:
        grayscale_attribution = attribution[0]  # на випадок якщо лише один канал

    # Нормалізація
    grayscale_attribution -= grayscale_attribution.min()
    if grayscale_attribution.max() != 0:
        grayscale_attribution /= grayscale_attribution.max()

    # Візуалізація
    original_resized = image.resize((224, 224))

    plt.figure(figsize=(6, 6))
    plt.imshow(original_resized)
    plt.imshow(grayscale_attribution, cmap='hot', alpha=0.5)
    plt.title(f"Attention Map - Label: {'Non-Smoker' if true_label == 0 else 'Smoker'}")
    plt.axis('off')
    plt.colorbar()
    plt.show()

# Приклад використання:
image_path = "/content/ChatGPT.png"  # Замініть на ваш шлях до зображення
true_label = 1  # 0 = Non-Smoker, 1 = Smoker

visualize_attention_map(model, image_path, true_label)

def visualize_attention_map(model, image_path, true_label, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()

    # Завантаження зображення
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Прогноз
    with torch.no_grad():
        output = torch.sigmoid(model(input_tensor)).item()
    predicted_class = 1 if output >= 0.5 else 0

    print(f"Predicted: {predicted_class} ({'Smoker' if predicted_class else 'Non-Smoker'}), Score: {output:.3f}")
    print(f"True label: {true_label} ({'Smoker' if true_label else 'Non-Smoker'})")

    # Integrated Gradients
    ig = IntegratedGradients(model)
    attribution = ig.attribute(input_tensor, target=None).squeeze().cpu().detach().numpy()

    # Усереднення по каналах (RGB → 1 канал)
    if attribution.ndim == 3:
        attribution = np.mean(attribution, axis=0)

    # Нормалізація атрибуції
    attribution -= np.min(attribution)
    attribution /= np.max(attribution) + 1e-8  # додавання епсилону для уникнення ділення на 0

    # Масштабування до 0-255
    heatmap = np.uint8(255 * attribution)

    # Кольорова мапа (COLORMAP_JET)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)

    # Оригінальне зображення (без трансформацій)
    image_resized = image.resize((224, 224))
    original_array = np.array(image_resized)

    # Переконаймося, що обидва зображення мають однакову форму
    if original_array.shape != heatmap_color.shape:
        heatmap_color = cv2.resize(heatmap_color, (original_array.shape[1], original_array.shape[0]))

    # Накладання
    overlayed = cv2.addWeighted(original_array, 0.5, heatmap_color, 0.5, 0)

    # Візуалізація
    plt.figure(figsize=(6, 6))
    plt.imshow(overlayed)
    plt.title(f"Attention Map - Label: {'Smoker' if true_label else 'Non-Smoker'}")
    plt.axis('off')
    plt.show()

# Приклад використання:
image_path = "/content/photo_2025-05-02_14-52-57.jpg"  # Замініть на ваш шлях до зображення
true_label = 1  # 0 = Non-Smoker, 1 = Smoker

visualize_attention_map(model, image_path, true_label)